In [1]:
import pandas as pd

In [4]:
df = pd.read_csv(r'mnit.txt', delimiter='\t')
df.head()

,"10/14/18, 08:04 - Diouf created group ""My Neck Is Throats"""
0,"10/14/18, 08:04 - You were added"
1,"8/23/19, 07:55 - Peter Moyo: Two albums droppe..."
2,"8/23/19, 07:56 - Peter Moyo: <Media omitted>"
3,"8/23/19, 08:09 - Tb Kamanga: Torrents. Uh oh"
4,"8/23/19, 08:10 - Tb Kamanga: 2 guys here strug..."


In [5]:
#df.to_csv('output.csv', index=False)


In [10]:
import pandas as pd
import re

# Load the raw text file
with open('mnit.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()

# Regex pattern to detect new message lines (e.g., 8/24/19, 07:55 - ...)
pattern = re.compile(r'^\d{1,2}/\d{1,2}/\d{2}, \d{1,2}:\d{2} - ')

# Combine lines into full messages
messages = []
current_message = ''

for line in lines:
    if pattern.match(line):
        if current_message:
            messages.append(current_message.strip())
        current_message = line.strip()
    else:
        current_message += ' ' + line.strip()

# Append the last message
if current_message:
    messages.append(current_message.strip())

# Parse messages into structured data
data = []

for msg in messages:
    try:
        date_time, rest = msg.split(' - ', 1)
        date_str, time_str = date_time.split(', ')
        if ': ' in rest:
            sender, message = rest.split(': ', 1)
        else:
            sender = None
            message = rest
        data.append([date_str, time_str, sender, message])
    except ValueError:
        continue  # Skip malformed lines

# Create DataFrame
df = pd.DataFrame(data, columns=['Date', 'Time', 'Sender', 'Message'])

# Convert to proper date and time formats
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%y').dt.date
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M').dt.time

# Combine into a single timestamp
df['Timestamp'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'].astype(str))

# Preview the result
df.head()




,Date,Time,Sender,Message,Timestamp
0,2018-10-14,08:04:00,None,"Diouf created group ""My Neck Is Throats""",2018-10-14 08:04:00
1,2018-10-14,08:04:00,None,You were added,2018-10-14 08:04:00
2,2019-08-23,07:55:00,Peter Moyo,Two albums dropped today,2019-08-23 07:55:00
3,2019-08-23,07:56:00,Peter Moyo,<Media omitted>,2019-08-23 07:56:00
4,2019-08-23,08:09:00,Tb Kamanga,Torrents. Uh oh,2019-08-23 08:09:00


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166705 entries, 0 to 166704
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   Date       166705 non-null  object        
 1   Time       166705 non-null  object        
 2   Sender     166666 non-null  object        
 3   Message    166705 non-null  object        
 4   Timestamp  166705 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(4)
memory usage: 6.4+ MB


In [12]:
df.to_csv('whatsapp_cleaned.csv', index=False)
